In [1]:
from geometry import *
from ngcotree import *

geo = geometryCoil()

mesh = Mesh(geo.GenerateMesh(maxh=0.05))
mesh.Refine()
mesh.Curve(2)

print("Meshing complete")
print(f"Number of elements : {mesh.ne}, Number of edges :{mesh.nedge}")
Draw(mesh,clipping={"x" : 0, "y":1, "z" : 0, "dist":0.07*max(min(1,1/splitting),0.5)})

Meshing complete
Number of elements : 26544, Number of edges :35559


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [2]:
jcf = 1e6*Normalize(CoefficientFunction((-y,x,0)))
nu0 = 1/(4e-7*np.pi)
nu  = mesh.MaterialCF({"core" : nu0/1000}, default = nu0)

## Discretization on the mesh elements
j = GridFunction(L2(mesh, dim = 3)); 
j.Set(jcf, definedon = mesh.Materials("coil")) # outside "coil", j is set to 0

Draw(j ,mesh,clipping={"z":-1, "dist":0},vectors = { "grid_size":100})

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [3]:
## FE space
dirichletLabels = "dOmega_back|dOmega_forward|dOmega_right|dOmega_left|dOmega_up|dOmega_bottom"
W1 = HCurl(mesh, dirichlet=dirichletLabels)

%matplotlib qt
newFreeDofs = CoTreeBitArray(mesh, W1, plot = True)

## Weak form
a = W1.TrialFunction()
astar = W1.TestFunction()
K = BilinearForm(curl(astar)*nu*curl(a)*dx) # same as previous K
f = LinearForm(astar*jcf*dx("coil")) 

## Assembly
with TaskManager():
    K.Assemble()
    f.Assemble()

directSolver = K.mat.Inverse(freedofs = newFreeDofs, inverse = "sparsecholesky")
solutionDirect = GridFunction(W1)

with TaskManager():
    solutionDirect.vec.data = directSolver * f.vec 
if np.sum(np.isnan(solutionDirect.vec[:].FV().NumPy())):
    print("... Failure : singular system !")
else:
    print("... Success!")
print(f"Are there NaN in the solution? {np.isnan(solutionDirect.vec[:].FV().NumPy())}")
Draw(curl(solutionDirect),mesh,clipping = {"x" : 0, "y" : 1, "z" : 0, "dist" : 0},vectors = { "grid_size":100})

... Success!
Are there NaN in the solution? [False False False ... False False False]


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [4]:
# fig = plt.figure(projection='3d')